In [2]:
from access_nri_intake.source import builders
from dask.distributed import Client
# Hide some warnings
# logging.basicConfig(level=logging.WARNING)

In [8]:
import intake

esm_datastore = intake.open_esm_datastore(
    "/g/data/ps29/nd0349/runs/access-om3/archive/mom6-cice6_iaf_rel/intake_esm_ds.json", 
    columns_with_iterables=[
            "variable",
            "variable_long_name",
            "variable_standard_name",
            "variable_cell_methods",
            "variable_units",
    ] # This is important
)

esm_datastore

,unique
filename,1141
file_id,6
path,1159
filename_timestamp,456
frequency,3
start_date,457
end_date,457
variable,200
variable_long_name,185
variable_standard_name,41


In [ ]:
client = Client(threads_per_worker = 1)
client

## Search for unique attributes

In [9]:
esm_datastore.unique().frequency
esm_datastore.search(frequency='1mon').unique().variable
# esm_datastore.unique().variable

['time',
 'time_bounds',
 'NCAT',
 'VGRDi',
 'VGRDs',
 'VGRDb',
 'VGRDa',
 'blkmask',
 'ANGLE',
 'ANGLET',
 'hi_m',
 'hs_m',
 'Tsfc_m',
 'aice_m',
 'uvel_m',
 'vvel_m',
 'uatm_m',
 'vatm_m',
 'sice_m',
 'fswup_m',
 'fswdn_m',
 'flwdn_m',
 'snow_ai_m',
 'rain_ai_m',
 'frzmlt_m',
 'scale_factor_m',
 'fswabs_ai_m',
 'albsni_m',
 'alvdr_ai_m',
 'alidr_ai_m',
 'alvdf_ai_m',
 'alidf_ai_m',
 'albice_m',
 'albsno_m',
 'flat_ai_m',
 'fsens_ai_m',
 'flwup_ai_m',
 'evap_ai_m',
 'Tair_m',
 'congel_m',
 'frazil_m',
 'snoice_m',
 'meltt_m',
 'melts_m',
 'meltb_m',
 'meltl_m',
 'fresh_ai_m',
 'fsalt_m',
 'fsalt_ai_m',
 'fhocn_ai_m',
 'fswthru_ai_m',
 'strairx_m',
 'strairy_m',
 'strtltx_m',
 'strtlty_m',
 'strcorx_m',
 'strcory_m',
 'strocnx_m',
 'strocny_m',
 'strintx_m',
 'strinty_m',
 'strength_m',
 'divu_m',
 'shear_m',
 'vort_m',
 'dvidtt_m',
 'dvidtd_m',
 'daidtt_m',
 'daidtd_m',
 'trsig_m',
 'ice_present_m',
 'fcondtop_ai_m',
 'fmeltt_ai_m',
 'sifb_m',
 'opening_m',
 'aicen_m',
 'vicen_m',
 'f

In [5]:
%%time
# esm_datastore.search(frequency='1mon',variable='aice_m').to_dataset_dict()

# ds = esm_datastore.search(variable="aice_m").search(frequency="1mon").to_dask()
esm_datastore_filtered = esm_datastore.search(
    variable=["aice_m"], frequency="1mon", require_all_on="path"
)

CPU times: user 4.27 s, sys: 33.4 ms, total: 4.3 s
Wall time: 4.3 s


In [10]:
%%time
xarray_open_kwargs = {"chunks": {"time": 150, "nj": -1, "ni": -1}}

ds = esm_datastore_filtered.to_dask(xarray_open_kwargs=xarray_open_kwargs).aice_m

ESMDataSourceError: Failed to load dataset with key='access_om3_cice_1mon_mean_XXXX_XX.1mon'
                 You can use `cat['access_om3_cice_1mon_mean_XXXX_XX.1mon'].df` to inspect the assets/files for this key.
                 

In [54]:
ds

<xarray.DataArray 'aice_m' (time: 600, nj: 300, ni: 360)> Size: 518MB
dask.array<concatenate, shape=(600, 300, 360), dtype=float64, chunksize=(1, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 5kB 1900-01-16 12:00:00 ... 1949-12-16 12:00:00
Dimensions without coordinates: nj, ni
Attributes:
    units:          1
    long_name:      ice area  (aggregate)
    cell_measures:  area: tarea
    cell_methods:   time: mean
    time_rep:       averaged

In [44]:
# %%time
# ds = esm_datastore.search(variable="aice_m").search(frequency="1mon").to_dask().aice_m
# esm_datastore_filtered.unique().variable
# esm_datastore_filtered
ds["aice_m"].encoding

{'dtype': dtype('float64'),
 'zlib': True,
 'szip': False,
 'zstd': False,
 'bzip2': False,
 'blosc': False,
 'shuffle': False,
 'complevel': 1,
 'fletcher32': False,
 'contiguous': False,
 'chunksizes': (1, 150, 180),
 'preferred_chunks': {'time': 1, 'nj': 150, 'ni': 180},
 'source': '/g/data/ps29/nd0349/spack/0.22/spack-release/mom6-cice6_configs/mom6-cice6_ryf_rel/archive/output000/access-om3.cice.1mon.mean.1900-01.nc',
 'original_shape': (1, 300, 360),
 'missing_value': 1e+30,
 '_FillValue': 1e+30,
 'coordinates': 'TLON TLAT time'}

In [4]:
ds["temp"].isel(time=-1, st_ocean=0).plot()

NameError: name 'ds' is not defined